In [ ]:
def build_bigram_model(text):
    # Text in Wörter aufteilen
    words = text.split()

    # Bigramm-Zählungen initialisieren
    bigram_counts = {}

    # Durch alle aufeinanderfolgenden Wortpaare iterieren
    for i in range(len(words) - 1):
        current_word = words[i]
        next_word = words[i + 1]

        # Wenn das aktuelle Wort noch nicht im Dictionary ist, füge es hinzu
        if current_word not in bigram_counts:
            bigram_counts[current_word] = {}

        # Erhöhe die Zählung für das Folgwort
        if next_word not in bigram_counts[current_word]:
            bigram_counts[current_word][next_word] = 1
        else:
            bigram_counts[current_word][next_word] += 1

    return bigram_counts

In [ ]:
model = build_bigram_model("Die Sonne scheint. Die Wolken ziehen vorbei. Die Sonne wärmt.")
print(model)

{'Die': {'Sonne': 2, 'Wolken': 1}, 'Sonne': {'scheint.': 1, 'wärmt.': 1}, 'scheint.': {'Die': 1}, 'Wolken': {'ziehen': 1}, 'ziehen': {'vorbei.': 1}, 'vorbei.': {'Die': 1}}


Die obrige Implementierung hat den Nachteil, dass Satzzeichen so behandelt werden als wären diese ein Teil des Wortes. In der einfachen Aufgabe muss Schrödinger diese entfernen.

In [ ]:
def build_bigram_model(text):
    # Satzzeichen als eigenes Wort erkennen, indem wir ein Leerzeichen voranstellen.
    for punct in ".,!?;:()[]{}\"'":
        text = text.replace(punct, f" {punct} ")

    # Text in Wörter aufteilen
    words = text.split()

    # Bigramm-Zählungen initialisieren
    bigram_counts = {}

    # Durch alle aufeinanderfolgenden Wortpaare iterieren
    for i in range(len(words) - 1):
        current_word = words[i]
        next_word = words[i + 1]

        # Wenn das aktuelle Wort noch nicht im Dictionary ist, füge es hinzu
        if current_word not in bigram_counts:
            bigram_counts[current_word] = {}

        # Erhöhe die Zählung für das Folgwort
        if next_word not in bigram_counts[current_word]:
            bigram_counts[current_word][next_word] = 1
        else:
            bigram_counts[current_word][next_word] += 1

    return bigram_counts

model = build_bigram_model("Die Sonne scheint. Die Wolken ziehen vorbei. Die Sonne wärmt.")
print(model)

{'Die': {'Sonne': 2, 'Wolken': 1}, 'Sonne': {'scheint': 1, 'wärmt': 1}, 'scheint': {'.': 1}, '.': {'Die': 2}, 'Wolken': {'ziehen': 1}, 'ziehen': {'vorbei': 1}, 'vorbei': {'.': 1}, 'wärmt': {'.': 1}}


In [ ]:
def calculate_bigram_probabilities(bigram_counts):
    bigram_probabilities = {}

    for current_word, next_words in bigram_counts.items():
        # Gesamtzahl der Vorkommen des aktuellen Wortes berechnen
        total_count = sum(next_words.values())

        # Dictionary für Wahrscheinlichkeiten initialisieren
        bigram_probabilities[current_word] = {}

        # Wahrscheinlichkeiten für jedes Folgwort berechnen
        for next_word, count in next_words.items():
            bigram_probabilities[current_word][next_word] = count / total_count

    return bigram_probabilities

print(calculate_bigram_probabilities(model))

{'Die': {'Sonne': 0.6666666666666666, 'Wolken': 0.3333333333333333}, 'Sonne': {'scheint': 0.5, 'wärmt': 0.5}, 'scheint': {'.': 1.0}, '.': {'Die': 1.0}, 'Wolken': {'ziehen': 1.0}, 'ziehen': {'vorbei': 1.0}, 'vorbei': {'.': 1.0}, 'wärmt': {'.': 1.0}}


In [ ]:
def predict_next_word(bigram_probabilities, current_word):
    if current_word not in bigram_probabilities:
        return "Unbekanntes Wort", 0.0

    next_words = bigram_probabilities[current_word]
    most_likely_word = max(next_words.items(), key=lambda x: x[1])

    return most_likely_word

print(predict_next_word(model, "Die"))

('Sonne', 2)


In [ ]:
def suggest_next_words(bigram_probabilities, current_word, n=3):
    if current_word not in bigram_probabilities:
        return [("Unbekanntes Wort", 0.0)]

    next_words = bigram_probabilities[current_word]
    sorted_words = sorted(next_words.items(), key=lambda x: x[1], reverse=True)

    return sorted_words[:n]
print(suggest_next_words(model, "Die"))

[('Sonne', 2), ('Wolken', 1)]


In [ ]:
def simple_autocomplete():
    # Beispieltext zum Trainieren
    training_text = """Künstliche Intelligenz ist ein faszinierendes Forschungsgebiet.
    Künstliche Neuronen bilden die Grundlage für neuronale Netze.
    Intelligenz kann in verschiedenen Formen auftreten.
    Python ist eine beliebte Programmiersprache für maschinelles Lernen.
    Programmiersprache Python wird oft für Data Science verwendet.
    Maschinelles Lernen ermöglicht Computern, aus Daten zu lernen."""

    # Bigramm-Modell bauen
    bigram_counts = build_bigram_model(training_text)
    bigram_probs = calculate_bigram_probabilities(bigram_counts)

    # Interaktive Schleife für Autovervollständigung
    print("Einfache Autovervollständigung (Beenden mit 'exit')")
    while True:
        user_input = input("Gib ein Wort ein: ")
        if user_input.lower() == 'exit':
            break

        suggestions = suggest_next_words(bigram_probs, user_input)
        print("Vorschläge:")
        for word, prob in suggestions:
            print(f"  {word} ({prob:.2f})")
simple_autocomplete()

Einfache Autovervollständigung (Beenden mit 'exit')
Gib ein Wort ein: exit


In [ ]:

def calculate_smoothed_probabilities(bigram_counts):
    smoothed_probabilities = {}

    # Erstelle ein Vokabular aller im Modell vorkommenden Wörter
    vocabulary = set()
    for current_word, next_words in bigram_counts.items():
        vocabulary.add(current_word)
        for next_word in next_words:
            vocabulary.add(next_word)

    vocab_size = len(vocabulary)

    # Für jedes Wort im Vokabular
    for current_word in vocabulary:
        # Dictionary für Wahrscheinlichkeiten initialisieren
        smoothed_probabilities[current_word] = {}

        # Gesamtzahl der Vorkommen des aktuellen Wortes + Vokabulargröße für Add-One-Smoothing
        total_count = sum(bigram_counts.get(current_word, {}).values()) + vocab_size

        # Für jedes mögliche Folgwort im Vokabular
        for next_word in vocabulary:
            # Zählung abrufen (0 wenn nicht gesehen)
            count = bigram_counts.get(current_word, {}).get(next_word, 0)
            # Geglättete Wahrscheinlichkeit berechnen
            smoothed_probabilities[current_word][next_word] = (count + 1) / total_count

    return smoothed_probabilities
print(calculate_smoothed_probabilities(model))

{'Wolken': {'Wolken': 0.1111111111111111, 'Die': 0.1111111111111111, 'wärmt': 0.1111111111111111, 'vorbei': 0.1111111111111111, '.': 0.1111111111111111, 'ziehen': 0.2222222222222222, 'scheint': 0.1111111111111111, 'Sonne': 0.1111111111111111}, 'Die': {'Wolken': 0.18181818181818182, 'Die': 0.09090909090909091, 'wärmt': 0.09090909090909091, 'vorbei': 0.09090909090909091, '.': 0.09090909090909091, 'ziehen': 0.09090909090909091, 'scheint': 0.09090909090909091, 'Sonne': 0.2727272727272727}, 'wärmt': {'Wolken': 0.1111111111111111, 'Die': 0.1111111111111111, 'wärmt': 0.1111111111111111, 'vorbei': 0.1111111111111111, '.': 0.2222222222222222, 'ziehen': 0.1111111111111111, 'scheint': 0.1111111111111111, 'Sonne': 0.1111111111111111}, 'vorbei': {'Wolken': 0.1111111111111111, 'Die': 0.1111111111111111, 'wärmt': 0.1111111111111111, 'vorbei': 0.1111111111111111, '.': 0.2222222222222222, 'ziehen': 0.1111111111111111, 'scheint': 0.1111111111111111, 'Sonne': 0.1111111111111111}, '.': {'Wolken': 0.1, 'Di

In [ ]:
def build_character_bigram_model(text):
    # Bigramm-Zählungen initialisieren
    char_bigram_counts = {}

    # Durch alle aufeinanderfolgenden Zeichenpaare iterieren
    for i in range(len(text) - 1):
        current_char = text[i]
        next_char = text[i + 1]

        # Wenn das aktuelle Zeichen noch nicht im Dictionary ist, füge es hinzu
        if current_char not in char_bigram_counts:
            char_bigram_counts[current_char] = {}

        # Erhöhe die Zählung für das Folgezeichen
        if next_char not in char_bigram_counts[current_char]:
            char_bigram_counts[current_char][next_char] = 1
        else:
            char_bigram_counts[current_char][next_char] += 1

    return char_bigram_counts

print(build_character_bigram_model("Die Sonne scheint. Die Wolken ziehen vorbei. Die Sonne wärmt."))

{'D': {'i': 3}, 'i': {'e': 4, 'n': 1, '.': 1}, 'e': {' ': 5, 'i': 2, 'n': 2, 'h': 1}, ' ': {'S': 2, 's': 1, 'D': 2, 'W': 1, 'z': 1, 'v': 1, 'w': 1}, 'S': {'o': 2}, 'o': {'n': 2, 'l': 1, 'r': 1}, 'n': {'n': 2, 'e': 2, 't': 1, ' ': 2}, 's': {'c': 1}, 'c': {'h': 1}, 'h': {'e': 2}, 't': {'.': 2}, '.': {' ': 2}, 'W': {'o': 1}, 'l': {'k': 1}, 'k': {'e': 1}, 'z': {'i': 1}, 'v': {'o': 1}, 'r': {'b': 1, 'm': 1}, 'b': {'e': 1}, 'w': {'ä': 1}, 'ä': {'r': 1}, 'm': {'t': 1}}


N-Gramme beliebiger Größe

In [ ]:
def build_ngram_model(text, n):
    words = text.split()
    ngram_counts = {}

    # Durch alle möglichen N-Gramme iterieren
    for i in range(len(words) - n + 1):
        # Die ersten n-1 Wörter sind der Kontext
        context = tuple(words[i:i+n-1])
        next_word = words[i+n-1]

        if context not in ngram_counts:
            ngram_counts[context] = {}

        if next_word not in ngram_counts[context]:
            ngram_counts[context][next_word] = 1
        else:
            ngram_counts[context][next_word] += 1

    return ngram_counts

def calculate_smoothed_ngram_probabilities(ngram_counts):
    smoothed_probabilities = {}

    # Erstelle ein Vokabular aller Wörter im Modell
    vocabulary = set()
    for context, next_words in ngram_counts.items():
        for next_word in next_words:
            vocabulary.add(next_word)
        # Auch Wörter aus dem Kontext zum Vokabular hinzufügen
        vocabulary.update(context)

    vocab_size = len(vocabulary)

    # Für jeden Kontext
    for context, next_words in ngram_counts.items():
        # Dictionary für Wahrscheinlichkeiten initialisieren
        smoothed_probabilities[context] = {}

        # Gesamtzahl der Vorkommen des Kontexts + Vokabulargröße für Add-One-Smoothing
        total_count = sum(next_words.values()) + vocab_size

        # Für jedes mögliche Folgwort im Vokabular
        for next_word in vocabulary:
            # Zählung abrufen (0 wenn nicht gesehen)
            count = next_words.get(next_word, 0)
            # Geglättete Wahrscheinlichkeit berechnen
            smoothed_probabilities[context][next_word] = (count + 1) / total_count

    return smoothed_probabilities


print(calculate_smoothed_ngram_probabilities(build_ngram_model("Die Sonne scheint. Die Wolken ziehen vorbei. Die Sonne wärmt.", 3)))

{('Die', 'Sonne'): {'Wolken': 0.1111111111111111, 'Die': 0.1111111111111111, 'wärmt.': 0.2222222222222222, 'scheint.': 0.2222222222222222, 'ziehen': 0.1111111111111111, 'vorbei.': 0.1111111111111111, 'Sonne': 0.1111111111111111}, ('Sonne', 'scheint.'): {'Wolken': 0.125, 'Die': 0.25, 'wärmt.': 0.125, 'scheint.': 0.125, 'ziehen': 0.125, 'vorbei.': 0.125, 'Sonne': 0.125}, ('scheint.', 'Die'): {'Wolken': 0.25, 'Die': 0.125, 'wärmt.': 0.125, 'scheint.': 0.125, 'ziehen': 0.125, 'vorbei.': 0.125, 'Sonne': 0.125}, ('Die', 'Wolken'): {'Wolken': 0.125, 'Die': 0.125, 'wärmt.': 0.125, 'scheint.': 0.125, 'ziehen': 0.25, 'vorbei.': 0.125, 'Sonne': 0.125}, ('Wolken', 'ziehen'): {'Wolken': 0.125, 'Die': 0.125, 'wärmt.': 0.125, 'scheint.': 0.125, 'ziehen': 0.125, 'vorbei.': 0.25, 'Sonne': 0.125}, ('ziehen', 'vorbei.'): {'Wolken': 0.125, 'Die': 0.25, 'wärmt.': 0.125, 'scheint.': 0.125, 'ziehen': 0.125, 'vorbei.': 0.125, 'Sonne': 0.125}, ('vorbei.', 'Die'): {'Wolken': 0.125, 'Die': 0.125, 'wärmt.': 0.125

Back-Off-Modelle

In [ ]:
def predict_with_backoff(trigram_model, bigram_model, unigram_model, context_words):
    if len(context_words) >= 2:
        # Trigram context
        context = tuple(context_words[-2:])
        if context in trigram_model and trigram_model[context]:
            return max(trigram_model[context].items(), key=lambda x: x[1])

    if len(context_words) >= 1:
        # Bigram context
        context = context_words[-1]
        if context in bigram_model and bigram_model[context]:
            return max(bigram_model[context].items(), key=lambda x: x[1])

    # Unigram (häufigstes Wort)
    return max(unigram_model.items(), key=lambda x: x[1])

# Die Schwierige Aufgabe
Erweitere die Backoff-Methode um Interpolation, bei der die Vorhersagen verschiedener N-Gramm-Modelle mit Gewichten kombiniert werden.

In [ ]:
def predict_with_interpolation(trigram_model, bigram_model, unigram_model, context_words, lambda1=0.7, lambda2=0.2, lambda3=0.1):
    # Wahrscheinlichkeiten initialisieren
    trigram_prob = {}
    bigram_prob = {}
    unigram_prob = {}

    if len(context_words) >= 2:
        # Trigram context
        context = tuple(context_words[-2:])
        if context in trigram_model and trigram_model[context]:
            trigram_prob = trigram_model[context]

    if len(context_words) >= 1:
        # Bigram context
        context = context_words[-1]
        if context in bigram_model and bigram_model[context]:
            bigram_prob = bigram_model[context]

    # Unigram (häufigstes Wort)
    unigram_prob = unigram_model.get((), {})

    # Interpolierte Wahrscheinlichkeiten berechnen
    interpolated_prob = {}
    for word in set(trigram_prob.keys()).union(bigram_prob.keys()).union(unigram_prob.keys()):
        interpolated_prob[word] = (
            lambda1 * trigram_prob.get(word, 0) +
            lambda2 * bigram_prob.get(word, 0) +
            lambda3 * unigram_prob.get(word, 0)
        )

    # Vorhersage basierend auf der höchsten interpolierten Wahrscheinlichkeit
    return max(interpolated_prob.items(), key=lambda x: x[1])

# Beispielaufruf der Funktion
training_text = """Künstliche Intelligenz ist ein faszinierendes Forschungsgebiet.
Künstliche Neuronen bilden die Grundlage für neuronale Netze.
Intelligenz kann in verschiedenen Formen auftreten.
Python ist eine beliebte Programmiersprache für maschinelles Lernen.
Programmiersprache Python wird oft für Data Science verwendet.
Maschinelles Lernen ermöglicht Computern, aus Daten zu lernen."""

trigram_model = calculate_smoothed_ngram_probabilities(build_ngram_model(training_text, 3))
bigram_model = calculate_smoothed_ngram_probabilities(build_ngram_model(training_text, 2))
unigram_model = calculate_smoothed_ngram_probabilities(build_ngram_model(training_text, 1))
context_words = ["spannendes", "Forschungsgebiet"]
prediction = predict_with_interpolation(trigram_model, bigram_model, unigram_model, context_words)
print(prediction)


('für', 0.0049382716049382715)


# Optimierungen

In [ ]:
# Effizientere Implementierung mit Defaultdict
from collections import defaultdict

def build_ngram_model_optimized(text, n):
    words = text.split()
    # Verschachtelte defaultdicts für automatische Initialisierung
    ngram_counts = defaultdict(lambda: defaultdict(int))

    # Durch alle möglichen N-Gramme iterieren
    for i in range(len(words) - n + 1):
        # Die ersten n-1 Wörter sind der Kontext
        context = tuple(words[i:i+n-1])
        next_word = words[i+n-1]

        # Mit defaultdict brauchen wir keine Existenzprüfung mehr
        ngram_counts[context][next_word] += 1

    return ngram_counts

def prune_ngram_model(ngram_model, min_count=2):
    pruned_model = defaultdict(lambda: defaultdict(int))

    for context, next_words in ngram_model.items():
        for word, count in next_words.items():
            if count >= min_count:
                pruned_model[context][word] = count

    return pruned_model